In [2]:
# 1 Importing packages and the functions.py
import os
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import getpass
import glob
import seaborn as sns
import functions
import lfp_pre_processing_functions
import power_functions
import coherence_functions
import spectrogram_plotting_functions
import plotting_styles
import scipy.stats
import mne_connectivity
import mne
importlib.reload(functions) #loads our custom made functions.py file
importlib.reload(spectrogram_plotting_functions)
importlib.reload(plotting_styles)

linestyle = plotting_styles.linestyles
colors = plotting_styles.colors




# 2 - Loading the data files


This code fetches the current 'user' by using getpass. Then it sets the basepath, loads the files and specifies the savepath. Note that the basepath, files and savepath need to be changed depending on where you have kept the files and where you want the results to be stored. In this case, I have set it up to be in a particular folder in my Dropbox account, which is stored locally.


In [3]:
#Fetch the current user
user= (getpass.getuser())
print("Hello", user)

if user == 'CPLab':
    base='D:\\Dropbox\\CPLab'
else:
    base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
#Set the basepath, savepath and load the data files
files = glob.glob(base+'\\all_data_mat_250825\\*.mat')
savepath = base+'\\results\\'
print("Base path:", base)
print("Save path:", savepath)
print(files)


all_bands_dict = {'total':[1,100], 'beta':[12,30], 'gamma':[30,80], 'theta':[4,12]}

Hello sinha
Base path: C:\Users\sinha\Dropbox\CPLab
Save path: C:\Users\sinha\Dropbox\CPLab\results\
['C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk1_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk3_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk5_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230529_dk6_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk1_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk3_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk5_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230531_dk6_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230609_dk1_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_250825\\20230609_dk3_BW_nocontext_day1.mat', 'C:\\Users\\

In [16]:
df = pd.read_csv(savepath + 'coh_beh_pre_dig_1000ms.csv')


In [17]:
# List of feature columns (from index 9 onwards)
feature_cols = df.columns[9:]

# Define a z-score function
def z_score(x):
    if x.std() == 0:
        return x - x.mean() # Avoid division by zero, just center it
    return (x - x.mean()) / x.std()

# Apply transform grouped by unique_id
df_normalized = df.copy()
df_normalized[feature_cols] = df.groupby('unique_id')[feature_cols].transform(z_score)

# Check the result - let's see means and stds for one group to verify
group_id = df['unique_id'].unique()[0]
subset_original = df[df['unique_id'] == group_id][feature_cols]
subset_norm = df_normalized[df_normalized['unique_id'] == group_id][feature_cols]

print(f"Group: {group_id}")
print("Original Means (head):")
print(subset_original.mean().head())
print("\nNormalized Means (should be ~0):")
print(subset_norm.mean().head())
print("\nNormalized Stds (should be ~1):")
print(subset_norm.std().head())

# Save to CSV so I can mention it (though user can't download, I can describe it)
df_normalized.to_csv(savepath+'normalized_neural_data.csv', index=False)

Group: dk1_BWnocontext20230609
Original Means (head):
beta_pre_dig              0.614021
beta_pli_pre_dig          0.399638
beta_plv_pre_dig          0.482995
beta_aon_power_pre_dig    0.018833
beta_vhp_power_pre_dig    0.018790
dtype: float64

Normalized Means (should be ~0):
beta_pre_dig              6.344132e-17
beta_pli_pre_dig          1.244045e-16
beta_plv_pre_dig         -1.714898e-16
beta_aon_power_pre_dig    7.335402e-17
beta_vhp_power_pre_dig    7.930164e-18
dtype: float64

Normalized Stds (should be ~1):
beta_pre_dig              1.0
beta_pli_pre_dig          1.0
beta_plv_pre_dig          1.0
beta_aon_power_pre_dig    1.0
beta_vhp_power_pre_dig    1.0
dtype: float64


In [ ]:

data_melted = pd.melt(data,
                      id_vars = ['rat', 'task', 'date', 'unique_id', 'trial', 'side','correct?', 'time_to_dig'],
                      value_vars=['beta_pre_dig', 'beta_pli_pre_dig',
       'beta_plv_pre_dig', 'beta_aon_power_pre_dig', 'beta_vhp_power_pre_dig',
       'gamma_pre_dig', 'gamma_pli_pre_dig', 'gamma_plv_pre_dig',
       'gamma_aon_power_pre_dig', 'gamma_vhp_power_pre_dig', 'theta_pre_dig',
       'theta_pli_pre_dig', 'theta_plv_pre_dig', 'theta_aon_power_pre_dig',
       'theta_vhp_power_pre_dig', 'theta+beta_pre_dig',
       'theta+beta_pli_pre_dig', 'theta+beta_plv_pre_dig',
       'theta+beta_aon_power_pre_dig', 'theta+beta_vhp_power_pre_dig',
       'total_pre_dig', 'total_pli_pre_dig', 'total_plv_pre_dig',
       'total_aon_power_pre_dig', 'total_vhp_power_pre_dig'],
        value_name='value', var_name='band_parameter')

def split_at(s, c, n):
    words = s.split(c)
    return c.join(words[:n]), c.join(words[n:])
def replace_pre_dig(s):
    if s == 'pre_dig':
        s='coh'
    else:
        s=s
    return s

def remove_pre_dig(s):
    if "_" in s:
        parameter = split_at(s, "_", 1)[0]
    else:
        parameter = "coh"
    return parameter

def add_power(s):
    if s == 'aon':
        s='aon_power'
    elif s == "vhp":
        s='aon_power'
    else:
        s=s
    return s

data_melted['band']=data_melted['band_parameter'].apply(lambda x : split_at(x,"_",1)[0])
data_melted['parameter']=data_melted['band_parameter'].apply(lambda x : split_at(x,"_",1)[1])

data_melted['parameter'] = data_melted['parameter'].apply(lambda x : replace_pre_dig(x))

data_melted['parameter'] = data_melted['parameter'].apply(lambda x : remove_pre_dig(x))
data_melted['parameter'] = data_melted['parameter'].apply(lambda x : add_power(x))

# Feature Engineering

## Normalizing data

In [ ]:
data_melted_rormalized = []

